# Gamepad

> This is the functionality for collecting gamepad inputs

In [ ]:
#| default_exp observation.gamepad
#| export
import time
import inputs

from algorithmic_gamer.utility.types import *


In [ ]:
#| export
def normalize_stick_event(event):
    # Normalize stick events to the range [-1, 1]
    value = event.state / 32767
    return value

def normalize_trigger_event(event):
    # Normalize trigger events to the range [0, 1]
    value = event.state / 255
    return value

`normalize_stick_event` and `normalize_trigger_event` convert the values from the gamepad to bounded normalized values.

In [ ]:
#| export
def process_stick_event(event, last_values, event_list, THRESHOLD, verbose):
    value = normalize_stick_event(event)
    # Check if the input has been recorded before
    if event.code in last_values:
        # If the input has been recorded before, check if the value has changed sufficiently
        if abs(value - last_values[event.code]) > THRESHOLD:
            # If the value has changed sufficiently, record the event
            event_list.append(GamepadEventObservation(timestamp=time.time(), code=event.code, state=value))
            last_values[event.code] = value
            if verbose:
                print(f'event.code{event.code}, event.state {value}')
    else:
        # If the input has not been recorded before, record the event
        event_list.append(GamepadEventObservation(timestamp=time.time(), code=event.code, state=value))
        last_values[event.code] = value
        if verbose:
            print(f'event.code{event.code}, event.state {value}')
    return last_values, event_list

`process_stick_event` processes stick events from the gamepad and appends the event to the `event_list` when change in stick value exceeds a defined `THRESHOLD`

In [ ]:
#| export
def process_trigger_event(event, last_values, event_list, THRESHOLD, verbose):
    value = normalize_trigger_event(event)
    # Check if the input has been recorded before
    if event.code in last_values:
        # If the input has been recorded before, check if the value has changed sufficiently
        if abs(value - last_values[event.code]) > THRESHOLD:
            # If the value has changed sufficiently, record the event
            event_list.append(GamepadEventObservation(timestamp=time.time(), code=event.code, state=value))
            last_values[event.code] = value
            if verbose:
                print(f'event.code{event.code}, event.state {value}')
    else:
        # If the input has not been recorded before, record the event
        event_list.append(GamepadEventObservation(timestamp=time.time(), code=event.code, state=value))
        last_values[event.code] = value
        if verbose:
            print(f'event.code{event.code}, event.state {value}')
    return last_values, event_list


`process_trigger_event` processes trigger events from the gamepad and appends the event to the `event_list` when change in trigger value exceeds a defined `THRESHOLD`

In [ ]:
#| export
def process_button_event(event, event_list, verbose):
    # Record button press events regardless of the change in value
    event_list.append(GamepadEventObservation(timestamp=time.time(), code=event.code, state=event.state))
    if verbose:
        print(f'event.code{event.code}, event.state {event.state}')
    return event_list

`process_button_event` appends button events to the `event_list`

In [ ]:
#| export
def record_gamepad_events(duration, THRESHOLD = .05, verbose=False):
    # Set up storage for the controller inputs
    event_list = []
    last_values = {}
    if verbose:
        print(f'record gamepad start:{time.time()}')
    end_time = time.time() + duration
    while time.time() < end_time:
        # Get the current controller inputs
        events = inputs.get_gamepad()
        
        if time.time() > end_time:#handles an issue where the while loop wont escape when waiting for inputs.get_gamepad()
            break
        for event in events:
            #if stick
            if event.code.startswith('ABS_Y') or event.code.startswith('ABS_X') or event.code.startswith('ABS_RY') or event.code.startswith('ABS_RX'):
                last_values, event_list = process_stick_event(event, last_values, event_list, THRESHOLD, verbose)
            #if trigger    
            elif event.code.startswith('ABS_Z') or event.code.startswith('ABS_RZ'):
                last_values, event_list = process_trigger_event(event, last_values, event_list, THRESHOLD, verbose)
            #if button    
            elif event.code.startswith('BTN_'):
                event_list = process_button_event(event, event_list, verbose)
        
    return event_list


`record_gamepad_events` collects button, stick and trigger inputs for a fixed period of time defined by `duration` in seconds

In [ ]:
#|eval: false
record_gamepad_events(3)

[GamepadEventObservation(timestamp=1674067371.6870863, code='ABS_X', state=0.013306070131534776),
 GamepadEventObservation(timestamp=1674067371.6870863, code='ABS_Y', state=-0.06494338816492203),
 GamepadEventObservation(timestamp=1674067372.7759886, code='ABS_X', state=-0.0445265053254799),
 GamepadEventObservation(timestamp=1674067372.783989, code='ABS_X', state=-0.3542283394878994),
 GamepadEventObservation(timestamp=1674067372.7919884, code='ABS_X', state=-0.6056398205511643),
 GamepadEventObservation(timestamp=1674067372.7919884, code='ABS_Y', state=0.0007324442274239326),
 GamepadEventObservation(timestamp=1674067372.7999895, code='ABS_X', state=-0.9486373485518967),
 GamepadEventObservation(timestamp=1674067372.7999895, code='ABS_Y', state=0.24866481521042513),
 GamepadEventObservation(timestamp=1674067372.8079884, code='ABS_X', state=-1.000030518509476),
 GamepadEventObservation(timestamp=1674067372.8639889, code='ABS_Y', state=0.19623401593066195),
 GamepadEventObservation(tim